In [ ]:
import os
import wfdb
import numpy as np
import h5py

# Cấu hình
WFDB_ROOT_DIR = r"D:\Python\data-ppg-ecg\data\data" # THAY ĐỔI ĐƯỜNG DẪN CỦA BẠN
H5_FILENAME = 'MIMIC_III_Raw_Waveforms.h5'
SEGMENT_DURATION_SEC = 10
SAMPLING_RATE = 125.0
SEGMENT_LENGTH = int(SEGMENT_DURATION_SEC * SAMPLING_RATE)
CHANNELS_TO_EXTRACT = ['II', 'ABP', 'PLETH', 'ART']

In [ ]:
def find_all_records(root_dir):
    """
    Quét thư mục gốc (ví dụ: 'data') để tìm tất cả các tên bản ghi WFDB
    theo cấu trúc thư mục con (ví dụ: 'p000020/3544749_0008').
    """
    all_record_names = []
    
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.hea'):
                # 1. Loại bỏ phần mở rộng .hea
                record_name_only = filename[:-4]
                
                # 2. Lấy đường dẫn tương đối từ thư mục gốc
                relative_path = os.path.relpath(dirpath, root_dir)
                
                # 3. Tạo tên bản ghi WFDB (Record Name)
                # Ví dụ: relative_path = 'p000020', record_name_only = '3544749_0008'
                full_wfdb_record_name = os.path.join(relative_path, record_name_only)
                
                # Đảm bảo dùng dấu '/' cho WFDB, ngay cả trên Windows
                all_record_names.append(full_wfdb_record_name.replace('\\', '/')) 
                
    return all_record_names

# --- GỌI HÀM ĐỌC DỮ LIỆU ĐIỀU CHỈNH ---

def extract_raw_segment(full_record_name, channel_names, root_dir):
    """
    Đọc dữ liệu waveform thô bằng cách sử dụng TÊN BẢN GHI VÀ ĐƯỜNG DẪN GỐC.
    """
    segments = []
    
    try:
        # wfdb.rdsamp sẽ kết hợp root_dir + full_record_name để tìm tệp
        sig, fields = wfdb.rdsamp(full_record_name, 
                                  channels=channel_names, 
                                  physical=False, 
                                  pn_dir=root_dir) # ⬅️ Sử dụng root_dir

        if sig is None:
            return segments
        
        # ... (Phần chia segment và gán nhãn tiếp tục như trước) ...
        
    except Exception as e:
        # print(f"Lỗi khi xử lý record {full_record_name}: {e}")
        pass
        
    return segments

# ----------------------------------------

# Bắt đầu chạy chương trình
MIMIC_RECORD_LIST = find_all_records(WFDB_ROOT_DIR)

print(f"Tìm thấy tổng cộng {len(MIMIC_RECORD_LIST)} bản ghi/segments để xử lý.")

In [ ]:
def extract_raw_segment(full_record_name, channel_names, root_dir):
    """
    Đọc dữ liệu waveform thô từ tên bản ghi đã được xây dựng bằng đường dẫn tương đối 
    và chia thành các segments.
    """
    segments = []
    
    try:
        # wfdb.rdsamp sử dụng record_name và tìm kiếm trong root_dir
        # LƯU Ý: Đối với MIMIC-III Waveform, mỗi tệp HEA thường là một segment duy nhất.
        sig, fields = wfdb.rdsamp(full_record_name, 
                                  channels=channel_names, 
                                  physical=False, 
                                  pb_dir=root_dir) # Dùng pb_dir cho đường dẫn gốc
        
        if sig is None:
            return segments

        # Lấy Subject ID để tạo nhãn giả
        # Ví dụ: p00/p000001/p000001_segment -> p000001
        subject_id_part = full_record_name.split('/')[1] 
        subject_id = subject_id_part[1:]

        # Chia tín hiệu thành các đoạn có độ dài cố định
        for start in range(0, sig.shape[0] - SEGMENT_LENGTH, SEGMENT_LENGTH):
            segment = sig[start:start + SEGMENT_LENGTH, :]
            segments.append((segment, get_label(subject_id))) # Lưu cùng với nhãn
            
    except Exception as e:
        # print(f"Lỗi khi xử lý record {full_record_name}: {e}")
        pass
        
    return segments

# Hàm tạo nhãn giả (như trước)
def get_label(subject_id):
    try:
        return 1 if int(subject_id) % 5 == 0 else 0
    except:
        return 0

In [ ]:
# Mảng chứa dữ liệu cuối cùng
ALL_RAW_SEGMENTS = []
ALL_LABELS = []

for full_record_name in MIMIC_RECORD_LIST:
    
    # Kết quả trả về là danh sách các cặp (segment_data, label)
    results = extract_raw_segment(full_record_name, CHANNELS_TO_EXTRACT, WFDB_ROOT_DIR)
    
    for segment, label in results:
        ALL_RAW_SEGMENTS.append(segment)
        ALL_LABELS.append(label)

X_final_raw = np.array(ALL_RAW_SEGMENTS, dtype=np.int16) 
Y_final = np.array(ALL_LABELS, dtype=np.int32)

# --- LƯU VÀO HDF5 ---
# Sử dụng hàm lưu HDF5 từ câu trả lời trước
def save_to_h5(X, Y, filename):
    if os.path.exists(filename):
        os.remove(filename)

    with h5py.File(filename, 'w') as f:
        f.create_dataset('X_raw', data=X, compression="gzip", compression_opts=9)
        f.create_dataset('Y_labels', data=Y, compression="gzip", compression_opts=9)
        f.attrs['data_shape'] = X.shape
        f.attrs['channels'] = ','.join(CHANNELS_TO_EXTRACT) 
    
    print(f"✅ Đã tạo HDF5 thành công: {filename}")
    print(f"Tổng số segments: {X.shape[0]}")
    
save_to_h5(X_final_raw, Y_final, H5_FILENAME)

In [ ]:
import os
import wfdb
import numpy as np
import h5py

# --- CẤU HÌNH ---
# THAY ĐỔI ĐƯỜNG DẪN NÀY ĐẾN THƯ MỤC GỐC CHỨA 'p000020'
# Ví dụ: nếu file .hea của bạn nằm ở C:\MIMIC\data\p000020\..., thì WFDB_ROOT_DIR là C:\MIMIC\data
WFDB_ROOT_DIR = r"D:\Python\data-ppg-ecg\data\data" # THAY ĐỔI ĐƯỜNG DẪN CỦA BẠN
H5_FILENAME = 'MIMIC_III_Raw_Waveforms.h5'
SEGMENT_DURATION_SEC = 10
SAMPLING_RATE = 125.0
SEGMENT_LENGTH = int(SEGMENT_DURATION_SEC * SAMPLING_RATE)
CHANNELS_TO_EXTRACT = ['II', 'ABP', 'PLETH', 'ART'] 

# --- HÀM TẠO NHÃN GIẢ (LABEL) ---
def get_label(subject_id):
    """Tạo nhãn giả dựa trên ID bệnh nhân để demo."""
    try:
        # Subject ID là phần số trong tên thư mục (ví dụ: '000020' từ 'p000020')
        return 1 if int(subject_id) % 5 == 0 else 0
    except:
        return 0

# --- HÀM 1: TÌM KIẾM BẢN GHI (WFDB Record List) ---
def find_all_records(root_dir):
    """
    Quét thư mục gốc WFDB để tìm và trả về tên bản ghi WFDB (Relative Path/Filename).
    Định dạng: 'p000020/3544749_0008'
    """
    all_record_names = []
    print(f"Bắt đầu quét thư mục: {os.path.abspath(root_dir)}")
    
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.hea'):
                # 1. Tên tệp không có đuôi mở rộng
                record_name_only = filename[:-4]
                
                # 2. Đường dẫn tương đối từ thư mục gốc
                relative_path = os.path.relpath(dirpath, root_dir)
                if not filename.endswith('n.hea') and not filename.endswith('layout.hea'):
                    # 3. Tên bản ghi WFDB hoàn chỉnh (dùng '/' cho WFDB)
                    full_wfdb_record_name = os.path.join(relative_path, record_name_only)
                    
                    # Thay thế dấu backslash (Windows) bằng forward slash (WFDB/Linux)
                    all_record_names.append(full_wfdb_record_name.replace('\\', '/')) 
                
    return all_record_names

# --- HÀM 2: TRÍCH XUẤT DỮ LIỆU THÔ (Raw Data Extraction) ---
def extract_raw_segment(full_record_name, channel_names, root_dir):
    """
    Đọc dữ liệu waveform và thay thế kênh bị thiếu bằng mảng zeros (zero padding).
    """
    segments_with_labels = []
    
    try:
        # Lấy Subject ID và Label
        subject_dir = full_record_name.split('/')[0] 
        subject_id = subject_dir[1:]
        label = get_label(subject_id)
        
        # BƯỚC 1: Đọc TẤT CẢ các kênh có sẵn trong tệp (KHÔNG dùng tham số 'channels')
        # Điều này tránh lỗi ValueError khi kênh không tồn tại
        sig_all, fields = wfdb.rdsamp(full_record_name, 
                                      pn_dir=root_dir,
                                      )
        
        if sig_all is None:
            return segments_with_labels

        # Lấy danh sách tên các kênh CÓ SẴN trong tệp này
        available_channels = fields['sig_name']
        
        # BƯỚC 2: Xây dựng mảng dữ liệu với Zero Padding
        num_samples = sig_all.shape[0]
        num_target_channels = len(channel_names)
        
        # Khởi tạo mảng dữ liệu cuối cùng với zeros
        combined_signal = np.zeros((num_samples, num_target_channels), dtype=np.int16)
        
        for i, target_channel in enumerate(channel_names):
            if target_channel in available_channels:
                # Kênh CÓ SẴN: Copy dữ liệu vào mảng kết hợp
                channel_index_in_file = available_channels.index(target_channel)
                combined_signal[:, i] = sig_all[:, channel_index_in_file]
            else:
                # Kênh BỊ THIẾU: Kênh i đã được điền zeros (mặc định)
                # print(f"  [Padding] Kênh '{target_channel}' bị thiếu trong {full_record_name}")
                pass 
                
        # BƯỚC 3: Chia đoạn (Segmentation) và Thu thập
        if num_samples < SEGMENT_LENGTH:
             return segments_with_labels
        
        for start in range(0, num_samples - SEGMENT_LENGTH + 1, SEGMENT_LENGTH):
            segment = combined_signal[start:start + SEGMENT_LENGTH, :]
            segments_with_labels.append((segment, label))
            
    except Exception as e:
        # Giữ lại in lỗi cho các vấn đề nghiêm trọng khác (không phải lỗi thiếu kênh)
        print(f"❌ Lỗi không xác định khi xử lý {full_record_name}: {e}")
        
    return segments_with_labels

# --- HÀM LƯU HDF5 (Đã được kiểm tra) ---
def save_to_h5(X, Y, filename):
    if os.path.exists(filename):
        os.remove(filename)

    with h5py.File(filename, 'w') as f:
        f.create_dataset('X_raw', data=X, compression="gzip", compression_opts=9)
        f.create_dataset('Y_labels', data=Y, compression="gzip", compression_opts=9)
        f.attrs['data_shape'] = X.shape
        f.attrs['channels'] = ','.join(CHANNELS_TO_EXTRACT) 
    
    print(f"✅ Đã tạo HDF5 thành công: {filename}")
    print(f"Tổng số segments: {X.shape[0]}")

# --- QUY TRÌNH CHÍNH ---
if __name__ == '__main__':
    MIMIC_RECORD_LIST = find_all_records(WFDB_ROOT_DIR)
    
    if not MIMIC_RECORD_LIST:
        print("\nFATAL ERROR: Danh sách bản ghi rỗng. Vui lòng kiểm tra lại WFDB_ROOT_DIR.")
    else:
        print(f"Tìm thấy {len(MIMIC_RECORD_LIST)} tệp .hea. Bắt đầu trích xuất...")

        ALL_RAW_SEGMENTS = []
        ALL_LABELS = []

        for full_record_name in MIMIC_RECORD_LIST:
            # Nhận danh sách các segments và nhãn từ một bản ghi
            results = extract_raw_segment(full_record_name, CHANNELS_TO_EXTRACT, WFDB_ROOT_DIR)
            
            for segment, label in results:
                ALL_RAW_SEGMENTS.append(segment)
                ALL_LABELS.append(label)

        X_final_raw = np.array(ALL_RAW_SEGMENTS, dtype=np.int16) 
        Y_final = np.array(ALL_LABELS, dtype=np.int32)

        # Kiểm tra cuối cùng trước khi lưu
        if X_final_raw.shape[0] > 0:
            save_to_h5(X_final_raw, Y_final, H5_FILENAME)
        else:
            print("\n❌ LỖI CUỐI CÙNG: Tổng số segments bằng 0 sau khi xử lý. Có thể tất cả các tệp đều thiếu kênh hoặc quá ngắn.")